# Summary
This notebook takes kodeoversikt.csv fetched from nin3-kode-api and merges with guids from drupal 'Bok1.xlsx'  
It merges these two into a dataframe that is then exported  
as csv to be imported into drupal update of langkode (out/gt_output_file.csv).

# Fetch kodeoversikt.csv into 2 dataframes, 1 for hovetype and one for grunntype

In [ ]:
import pandas as pd

# Read the CSV file into a dataframe
nin3koder = pd.read_csv('inndata/kodeoversikt.csv', sep=';')

# Display the dataframe
df_ht = nin3koder[nin3koder['Klasse'].isin(['Hovedtype'])]
df_gt = nin3koder[nin3koder['Klasse'].isin(['Grunntype'])]
print("Hovedtyper numrows:")
display(df_ht.info())
print("Grunntyper numrows:")
display(df_gt.info())



# Fetch data from drupal with the guids

In [ ]:
import pandas as pd

"""Bok1.xlsx is an export from the drupal database. 
It contains nin3 objects from drupal, guid, title and more"""
dp_df = pd.read_excel('inndata/Bok1.xlsx') #NiN Data from drupal-query
dp_df.dropna(subset=['feeds_item_guid'], inplace=True)# Remove rows with no feeds_item_guid
dp_df = dp_df[~dp_df['feeds_item_guid'].str.contains('-M005-|-M020-|-M050-')]# Remove rows that is 'kartleggingsenheter'
dp_df.head(3)

## Merging hovedtype with drupal guid

In [ ]:
# Left join the two dataframes on the 'Navn' column, left dataframe is df_ht(hovedtype from NiNcode API)
merged_ht = pd.merge(df_ht, dp_df, left_on='Navn', right_on='node_title', how='left')
merged_ht_unique = merged_ht.sort_values(by='nid', ascending=False).drop_duplicates(subset='Langkode', keep='first')
merged_ht_sorted = merged_ht_unique.sort_values(by='Langkode')
display(merged_ht_sorted.head(10))
# Save the dataframe to a CSV file that shows the relation between drupal<>nin3 (mapping of drupal-guid against NiN-code)
merged_ht_sorted.to_csv('out/merged_ht.csv', index=False, sep=';')


# Merging grunntype with drupal guid

In [ ]:
merged_gt = pd.merge(df_gt, dp_df, left_on='Navn', right_on='node_title', how='left')
merged_gt_unique = merged_gt.sort_values(by='nid', ascending=False).drop_duplicates(subset='Langkode', keep='first')
merged_gt_sorted = merged_gt_unique.sort_values(by='Langkode')
display(merged_gt_sorted.head(10))
# Save the dataframe to a CSV file that shows the relation between drupal<>nin3 (mapping of drupal-guid against NiN-code)
merged_gt_sorted.to_csv('out/merged_gt.csv', index=False, sep=';')

# Importfile (langkodeupdate) for hovedtype
- creates a importer file for drupal to update langkode on klassetype:Hovedtype, so that
langkode in drupal match with langkode in 'nin3kodeapi'.

In [ ]:
new_df = merged_ht_sorted[['Navn', 'feeds_item_guid', 'Langkode']].copy()
#new_df.columns = ['Title', 'GUID', 'Tags', 'LiteralValue']
new_df['Tags'] = 'NiN|NIN3|Hovedtype'
new_df['LiteralTags'] = 'Code'
new_df.rename(columns={'Navn': 'Title', 'feeds_item_guid': 'GUID', 'Langkode': 'LiteralValue'}, inplace=True)

# Set LiteralValue as the last column
literal_value = new_df.pop('LiteralValue')
new_df.insert(len(new_df.columns), 'LiteralValue', literal_value)
display(new_df.head(10))
new_df.to_csv('out/ht_output_file.csv', sep='\t', index=False)



# Importfile (langkodeupdate) for grunntype
- creates a importer file for drupal to update langkode on klassetype:Grunntype, so that
langkode in drupal match with langkode in 'nin3kodeapi'.

In [ ]:
new_gt_df = merged_gt_sorted[['Navn', 'feeds_item_guid', 'Langkode']].copy()
#new_df.columns = ['Title', 'GUID', 'Tags', 'LiteralValue']
new_gt_df['Tags'] = 'NiN|NIN3|Grunntype'
new_gt_df['LiteralTags'] = 'Code'
new_gt_df.rename(columns={'Navn': 'Title', 'feeds_item_guid': 'GUID', 'Langkode': 'LiteralValue'}, inplace=True)


# Set LiteralValue as the last column
literal_value = new_gt_df.pop('LiteralValue')
new_gt_df.insert(len(new_gt_df.columns), 'LiteralValue', literal_value)
display(new_gt_df.head(10))
new_gt_df.to_csv('out/gt_output_file.csv', sep='\t', index=False)